# 1. Stuff method

In [1]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain

In [10]:
prompt_template=""" Write a concise summary for the following:
                    {text}
                    CONCISE SUMMARY: """

prompt=PromptTemplate.from_template(prompt_template)

In [3]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.llms import Ollama
from langchain.document_loaders import WebBaseLoader

In [4]:
loader=WebBaseLoader(web_path="https://lilianweng.github.io/posts/2023-06-23-agent/")
docs=loader.load()

llm=Ollama(model="llama3")

In [11]:
llm_chain=LLMChain(llm=llm,prompt=prompt)

In [12]:
stuff_chain=StuffDocumentsChain(llm_chain=llm_chain,document_variable_name="text")

In [13]:
print(stuff_chain.invoke(docs)["output_text"])

Here is a concise summary:

The post discusses the challenges and limitations of building LLM-powered autonomous agents. The main challenges include:

1. Finite context length, which limits the inclusion of historical information and detailed instructions.
2. Limited long-term planning and task decomposition, making it difficult to adjust plans when faced with unexpected errors.
3. Unreliability of natural language interfaces, which may lead to formatting errors or rebellious behavior.

The post also references various papers and studies on topics such as chain-of-thought prompting, hindsight alignment, and optimal planning proficiency.


In [15]:
#load_summarize_chain(llm, chain_type="stuff").invoke(docs)["output_text"] this all can be done using this

# 2. Map-Reduce method

In [29]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.text_splitter import CharacterTextSplitter

In [17]:
map_template="""The following is a set of documents.
                {docs}
                Based on the list identify their main themes. """
map_prompt=PromptTemplate.from_template(map_template)
map_llm_chain=LLMChain(llm=llm,prompt=map_prompt)

In [19]:
reduce_template="""The following is set of summaries:
{docs}
Take these and distill it into a final, consolidated summary of the main themes."""

reduce_prompt=PromptTemplate.from_template(reduce_template)

In [20]:
reduce_chain=LLMChain(llm=llm,prompt=reduce_prompt)

In [21]:
combine_document_chain=StuffDocumentsChain(llm_chain=reduce_chain,document_variable_name="docs")

In [35]:
reduce_documents_chain=ReduceDocumentsChain(combine_documents_chain=combine_document_chain,
                                            collapse_documents_chain=combine_document_chain, token_max=2000)

In [36]:
map_reduce_chain=MapReduceDocumentsChain(llm_chain=map_llm_chain,reduce_documents_chain=reduce_documents_chain, 
                                        document_variable_name='docs', return_intermediate_steps=False)

In [37]:
text_splitter=CharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000,chunk_overlap=0)

In [38]:
split_docs=text_splitter.split_documents(docs)

In [ ]:
result=map_reduce_chain.invoke(split_docs)

In [ ]:
print(result["output_text"])